In [7]:
import requests

In [8]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [9]:
data = requests.get(standings_url)

In [10]:
from bs4 import BeautifulSoup

In [11]:
soup = BeautifulSoup(data.text)
standings_table = soup.select('table.stats_table')[0]
links = standings_table.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if '/squads/' in l]

In [12]:
team_urls = [f"https://fbref.com{l}" for l in links]

In [13]:
data = requests.get(team_urls[5])

In [14]:
import pandas as pd
matches = pd.read_html(data.text, match="Scores & Fixtures")[0]

C:\Users\VICTUS\AppData\Local\Temp\ipykernel_7248\4209044294.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]


In [15]:
soup = BeautifulSoup(data.text)
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'all_comps/shooting/' in l]

In [16]:
data = requests.get(f"https://fbref.com{links[0]}")

In [17]:
shooting = pd.read_html(data.text, match="Shooting")[0]

C:\Users\VICTUS\AppData\Local\Temp\ipykernel_7248\2517714723.py:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(data.text, match="Shooting")[0]


In [18]:
shooting.head()

For Manchester United                                                        \
                   Date   Time            Comp        Round  Day Venue Result   
0            2023-08-14  20:00  Premier League  Matchweek 1  Mon  Home      W   
1            2023-08-19  17:30  Premier League  Matchweek 2  Sat  Away      L   
2            2023-08-26  15:00  Premier League  Matchweek 3  Sat  Home      W   
3            2023-09-03  16:30  Premier League  Matchweek 4  Sun  Away      L   
4            2023-09-16  15:00  Premier League  Matchweek 5  Sat  Home      L   

                              ... Standard               Expected       \
    GF   GA         Opponent  ...     Dist   FK PK PKatt       xG npxG   
0  1.0  0.0           Wolves  ...     14.7  0.0  0     0      2.2  2.2   
1  0.0  2.0        Tottenham  ...     15.6  1.0  0     0      2.1  2.1   
2  3.0  2.0  Nott'ham Forest  ...     20.3  0.0  1     1      2.8  2.0   
3  1.0  3.0          Arsenal  ...     16.5  0.0  0     0      0.9  0.9   
4  1.0  3.0         Brighton  ...     18.6  1.0  0     0      1.0  1.0   

                       Unnamed: 25_level_0  
  npxG/Sh G-xG np:G-xG        Match Report  
0    0.15 -1.2    -1.2        Match Report  
1    0.09 -2.1    -2.1        Match Report  
2    0.12  0.2     0.0        Match Report  
3    0.09  0.1     0.1        Match Report  
4    0.07  0.0     0.0        Match Report  

[5 rows x 26 columns]

In [19]:
shooting.columns = shooting.columns.droplevel()

In [20]:
team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")

In [21]:
team_data.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Formation,Referee,Match Report,Notes,Sh,SoT,Dist,FK,PK,PKatt
0,2023-08-14,20:00,Premier League,Matchweek 1,Mon,Home,W,1.0,0.0,Wolves,...,4-1-4-1,Simon Hooper,Match Report,NaN,15,2,14.7,0.0,0,0
1,2023-08-19,17:30,Premier League,Matchweek 2,Sat,Away,L,0.0,2.0,Tottenham,...,4-2-3-1,Michael Oliver,Match Report,NaN,22,6,15.6,1.0,0,0
2,2023-08-26,15:00,Premier League,Matchweek 3,Sat,Home,W,3.0,2.0,Nott'ham Forest,...,4-2-3-1,Stuart Attwell,Match Report,NaN,17,8,20.3,0.0,1,1
3,2023-09-03,16:30,Premier League,Matchweek 4,Sun,Away,L,1.0,3.0,Arsenal,...,4-2-3-1,Anthony Taylor,Match Report,NaN,10,2,16.5,0.0,0,0
4,2023-09-16,15:00,Premier League,Matchweek 5,Sat,Home,L,1.0,3.0,Brighton,...,4-1-2-1-2◆,Jarred Gillett,Match Report,NaN,14,4,18.6,1.0,0,0


In [22]:
years = list(range(2024, 2022, -1))
all_matches = []

In [23]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [24]:
for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]
    
    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"
    
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        shooting = pd.read_html(data.text, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()
        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
        except ValueError:
            continue
        team_data = team_data[team_data["Comp"] == "Premier League"]
        
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        

C:\Users\VICTUS\AppData\Local\Temp\ipykernel_7248\3542750996.py:16: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
C:\Users\VICTUS\AppData\Local\Temp\ipykernel_7248\3542750996.py:21: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(data.text, match="Shooting")[0]
C:\Users\VICTUS\AppData\Local\Temp\ipykernel_7248\3542750996.py:16: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
C:\Users\VICTUS\AppData\Local\Temp\ipykernel_7248\3542750996.py:21: FutureWarning: Passing literal 

In [25]:
len(all_matches)

40

In [26]:
match_df = pd.concat(all_matches)

In [27]:
match_df.columns = [c.lower() for c in match_df.columns]

In [28]:
match_df

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
1,2023-08-12,12:30,Premier League,Matchweek 1,Sat,Home,W,2,1,Nott'ham Forest,...,Match Report,NaN,15.0,7.0,19.1,0.0,0,0,2024,Arsenal
2,2023-08-21,20:00,Premier League,Matchweek 2,Mon,Away,W,1,0,Crystal Palace,...,Match Report,NaN,13.0,2.0,16.4,0.0,1,1,2024,Arsenal
3,2023-08-26,15:00,Premier League,Matchweek 3,Sat,Home,D,2,2,Fulham,...,Match Report,NaN,18.0,9.0,13.8,0.0,1,1,2024,Arsenal
4,2023-09-03,16:30,Premier League,Matchweek 4,Sun,Home,W,3,1,Manchester Utd,...,Match Report,NaN,17.0,5.0,15.0,0.0,0,0,2024,Arsenal
5,2023-09-17,16:30,Premier League,Matchweek 5,Sun,Away,W,1,0,Everton,...,Match Report,NaN,13.0,4.0,17.4,0.0,0,0,2024,Arsenal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42,2023-04-30,14:00,Premier League,Matchweek 34,Sun,Away,L,1,3,Newcastle Utd,...,Match Report,NaN,4.0,3.0,17.3,0.0,0,0,2023,Southampton
43,2023-05-08,20:00,Premier League,Matchweek 35,Mon,Away,L,3,4,Nott'ham Forest,...,Match Report,NaN,18.0,4.0,14.0,0.0,1,1,2023,Southampton
44,2023-05-13,15:00,Premier League,Matchweek 36,Sat,Home,L,0,2,Fulham,...,Match Report,NaN,5.0,1.0,24.2,0.0,0,0,2023,Southampton
45,2023-05-21,14:00,Premier League,Matchweek 37,Sun,Away,L,1,3,Brighton,...,Match Report,NaN,5.0,1.0,13.8,1.0,0,0,2023,Southampton


In [30]:
match_df.to_csv("matches.csv")